In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf

# 웹 드라이버 초기화
url = "https://www.melon.com/"
driver = webdriver.Chrome()
driver.get(url)

search_term = input("검색할 음악제목을 입력하세요(한국노래만, 19금 노래 제외)")

search_box = driver.find_element(By.XPATH, "//[@id='top_search']")
search_box.send_keys(search_term)
search_box.send_keys(Keys.RETURN)

first_result = driver.find_element(By.XPATH, "//[@id='frm_searchSong']/div/table/tbody/tr[1]/td[3]/div/div/a[1]/span")
first_result.click()

# 가사 정보가 있는 요소 식별
lyrics_element = driver.find_element(By.XPATH, "//*[@id='d_video_summary']")

# 가사 텍스트 추출
lyrics_text = lyrics_element.text
print(lyrics_text)

# KoBERT 모델 로드
model_name = "beomi/kcbert-base"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 가사 텍스트를 KoBERT 입력 형식에 맞게 토큰화
inputs = tokenizer(lyrics_text, return_tensors='tf', padding=True, truncation=True)

# 감정 분석을 수행
with tf.device('CPU:0'):  # 또는 GPU를 지정할 수 있습니다.
    logits = model(inputs).logits

predicted_class = tf.argmax(logits, axis=1).numpy()[0]

# 감정 클래스에 따라 결과 출력
if predicted_class == 0:
    print("부정적 감정")
elif predicted_class == 1:
    print("중립적 감정")
else:
    print("긍정적 감정")

# 웹 드라이버 종료
driver.close()